<a href="https://colab.research.google.com/github/Alex-Ukraine/tf2-fcc/blob/master/tenth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
%tensorflow_version 2.x
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [128]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [129]:
#from google.colab import files
#path_to_file = list(files.upload().keys())[0]

In [130]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [131]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [132]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [133]:
# lets look at how part of our text is encoded
print("Text: ", text[:13])
print("Encoded: ", text_to_int(text[:13]))

Text:  First Citizen
Encoded:  [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [134]:
def int_to_text(ints):
    try:
        ints = ints.numpy()
    except:
        pass
    return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [135]:
seq_length = 100 # length of sequence for a training example
examples_per_epochs = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [136]:
sequence = char_dataset.batch(seq_length+1, drop_remainder=True)

In [137]:
def split_input_target(chunk): # for the example: hello
    input_text = chunk[:-1] # hell
    target_text = chunk[1:] # ello
    return input_text, target_text # hell, ello

dataset = sequence.map(split_input_target) # we use map to applythe above function to every entry

In [138]:
for x, y in dataset.take(2):
    print("\n\nEXAMPLE\n")
    print("INPUT")
    print(int_to_text(x))
    print("\nOUTPUT")
    print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [139]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [140]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                             return_sequences=True,
                             stateful=True,
                             recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (64, None, 256)           16640     
_________________________________________________________________
lstm_12 (LSTM)               (64, None, 1024)          5246976   
_________________________________________________________________
dense_12 (Dense)             (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [141]:
for input_example_batch, target_example_batch in data.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [142]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-1.88980950e-03  4.21602838e-03  1.92417216e-03 ...  2.10154243e-03
   -9.24142660e-04 -4.14139219e-03]
  [ 7.96040706e-03  4.51025134e-03 -4.84186923e-03 ...  4.74685617e-03
   -6.28145062e-04 -4.44870116e-03]
  [ 1.41864810e-02  5.46415104e-03 -9.77463461e-03 ...  6.23303885e-03
   -1.23073661e-03 -4.69120871e-03]
  ...
  [ 2.66432669e-03  2.71357666e-03 -5.68104582e-03 ...  8.21688492e-03
    1.06366547e-02 -1.16191506e-02]
  [-2.20804336e-03  8.05995520e-03 -3.94251896e-03 ...  1.11442106e-02
    3.33956070e-03 -1.08396355e-02]
  [ 8.30221688e-06  6.30343333e-03 -3.69537098e-04 ...  9.20815486e-03
   -3.54917976e-03 -1.09574925e-02]]

 [[ 9.21703316e-03  1.62601541e-03 -7.01514445e-03 ...  2.90072733e-03
   -8.38655804e-04 -1.26500707e-03]
  [ 1.50495768e-02  3.49429017e-03 -1.20111248e-02 ...  4.62927856e-03
   -1.95287750e-03 -2.00447324e-03]
  [ 7.57598691e-03  2.54156068e-03 -1.33437682e-02 ...  3.08222114e-03
   -5.30490885e-03 -2.75685103e-03]
  ...
  [-3.014

In [143]:
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[-1.88980950e-03  4.21602838e-03  1.92417216e-03 ...  2.10154243e-03
  -9.24142660e-04 -4.14139219e-03]
 [ 7.96040706e-03  4.51025134e-03 -4.84186923e-03 ...  4.74685617e-03
  -6.28145062e-04 -4.44870116e-03]
 [ 1.41864810e-02  5.46415104e-03 -9.77463461e-03 ...  6.23303885e-03
  -1.23073661e-03 -4.69120871e-03]
 ...
 [ 2.66432669e-03  2.71357666e-03 -5.68104582e-03 ...  8.21688492e-03
   1.06366547e-02 -1.16191506e-02]
 [-2.20804336e-03  8.05995520e-03 -3.94251896e-03 ...  1.11442106e-02
   3.33956070e-03 -1.08396355e-02]
 [ 8.30221688e-06  6.30343333e-03 -3.69537098e-04 ...  9.20815486e-03
  -3.54917976e-03 -1.09574925e-02]], shape=(100, 65), dtype=float32)


In [144]:
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[-1.8898095e-03  4.2160284e-03  1.9241722e-03  5.5619079e-04
 -4.6710325e-03 -2.3478572e-03 -4.6334062e-03  1.6653999e-03
 -3.3952617e-03 -8.3906687e-04  3.9735548e-03 -2.2856773e-04
 -1.3597731e-03  3.4830486e-04  1.2246600e-03 -3.7741801e-03
  1.3225643e-03  2.4787937e-03 -1.0560887e-03 -5.0417408e-03
 -2.4862713e-03  1.3487051e-03  3.3760443e-07  4.5950133e-03
 -9.7585684e-03 -3.0667144e-03  4.3581659e-03 -3.2974784e-03
  2.8941752e-03  1.2858654e-03 -3.1471027e-03 -1.3372555e-03
  4.4828015e-03  2.7710549e-03  1.3535117e-03 -2.1141791e-04
 -1.2082459e-03  5.3987926e-05  1.2933003e-03  2.7844859e-03
  1.4976857e-03  4.2956602e-04  5.1661162e-03  1.0562108e-03
 -7.8648608e-04  2.9293110e-03  2.0068814e-03 -1.3638997e-03
  3.8136018e-04 -2.4778452e-03 -1.2699430e-03  1.6433729e-03
  3.1934297e-03 -1.9582538e-03  2.7963985e-03  7.5692562e-03
  4.7550676e-03  2.0645200e-03 -3.7415703e-03 -1.5857135e-03
  2.6691160e-03  1.0068843e-03  2.1015424e-03 -9.2414266e-04
 -4.141392

In [145]:
sampled_indices = tf.random.categorical(pred, num_samples=1)

sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars

"xjq;ryLeZ\nOszoCgoEcYCNgcvxOHWrey:L'-etcKxMhLkcsFClSd!cq'vvcnPTN?WklDoZYm$PUpmPFHm3m33eD$Fn3gvmE,FZIP"

In [146]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [147]:
model.compile(optimizer='adam', loss=loss)

In [148]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [149]:
history = model.fit(data, epochs=1, callbacks=[checkpoint_callback])

172/172 [==============================] - 13s 77ms/step - loss: 2.6106


In [150]:
#model.save_weights("./training_checkpoints/ckpt_10.h5")

In [151]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [152]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [153]:
#checkpoint_num = 10
#model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
#model.build(tf.TensorShape([1, None]))

In [154]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 800

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    inpur_eval = tf.expand_dims(input_eval, 0)

    # Empty string store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted character as the next input tj the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [155]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo


IndexError: ignored